In [2]:
!pip install librosa
!pip install torch

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 260 kB 3.3 MB/s            
     |████████████████████████████████| 64 kB 2.4 MB/s             
     |████████████████████████████████| 3.7 MB 26.3 MB/s            
     |████████████████████████████████| 1.3 MB 89.4 MB/s            
     |████████████████████████████████| 242 kB 109.5 MB/s            
     |████████████████████████████████| 13.5 MB 60.4 MB/s            
     |████████████████████████████████| 308 kB 107.2 MB/s            
     |████████████████████████████████| 43.9 MB 99.9 MB/s            
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 888.0 MB 7.4 kB/s             
     |████████████████████████████████| 199 kB 23.4 MB/s            
     |████████████████████████████████| 44 kB 1.6 MB/s             
     |████████████████████████████████| 89 kB 6.2 MB/s             
     |████████

In [3]:
! git config --global credential.helper store


In [ ]:
! git push origin main


Username for 'https://github.com': 

In [ ]:
MP2024

In [1]:
import librosa
import torch

target_sample_rate = 24000

input_audio, orig_sample_rate = librosa.load("justin-master-class.wav")
input_audio = librosa.resample(
    input_audio, orig_sr=orig_sample_rate, target_sr=target_sample_rate
)
input_audio = torch.Tensor(input_audio)

ModuleNotFoundError: No module named 'librosa'

In [ ]:
input_audio.shape

In [ ]:
import numpy as np
from IPython.display import Audio

Audio(input_audio, rate=target_sample_rate)

In [ ]:
input_audio.shape

In [ ]:
from transformers import MimiModel


model = MimiModel.from_pretrained("kyutai/mimi")

with torch.no_grad():
    tokens = model.encode(input_audio[None, None, :]).audio_codes

In [ ]:
tokens = tokens.flip(-1)

In [ ]:
with torch.no_grad():
    embeddings = model.quantizer.decode(tokens)

    # Do weird things here
    # bias = torch.randn(1, embeddings.shape[1], 1) * 3
    # embeddings += bias

    # noise = torch.randn(1, embeddings.shape[1], embeddings.shape[2]) * 3
    # embeddings += noise

    # embeddings *= -1

    decoder_outputs = model.decoder_transformer(
        model.upsample(embeddings).transpose(1, 2), return_dict=False
    )
    embeddings = decoder_outputs[0].transpose(1, 2)
    audio_values = model.decoder(embeddings)

In [ ]:
import numpy as np
from IPython.display import Audio

Audio(audio_values.numpy()[0], rate=target_sample_rate)